## A minimalist example for recovering sparse graphs using mGLAD

Fitting meta-GLAD on a batch of erdos-renyi random sparse graphs with samples obtained from a corresponding multivariate Gaussian distribution

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/dagM


In [2]:
import torch
torch.__version__

'1.9.0'

### Create Synthetic data

In [3]:
from scripts import main
# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, trueTheta = main.getGLADdata(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=2
)

Smallest eval = 0.09999999999999981
Smallest eval = 0.10000000000000114
TrueTheta:
 (tensor([[[1.8740, 0.0000, 0.9087, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.0000, 1.8740, 0.7450, 0.6693, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.9087, 0.7450, 1.8740, 0.0000, 0.0000, 0.0000, 0.7367, 0.0000,
          0.0000, 0.7409],
         [0.0000, 0.6693, 0.0000, 1.8740, 0.0000, 0.0000, 0.0000, 0.7662,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.8740, 0.0000, 0.8100, 0.5383,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.8740, 0.0000, 0.9144,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.7367, 0.0000, 0.8100, 0.0000, 1.8740, 0.0000,
          0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.7662, 0.5383, 0.9144, 0.0000, 1.8740,
          0.8584, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8584,
          1.8740, 0.0000],
      

### Running the GLAD-Meta algorithm
Until Convergence:  

    1. Initialize learnable GLAD parameters
    2. Run the GLAD model
    3. Get the meta-loss
    4. Backprop

In [4]:
# main source file for meta-GLAD

# Helper functions for DAG Meta
from scripts.glad.glad_model import glad_params
from scripts.glad import glad
from scripts.main import lossGLADmeta
from scripts.utils import prepare_data
from scripts.utils.metrics import reportMetrics

from pprint import pprint

def initGLAD():
    """
    Initialize the GLAD model parameters and the optimizer
    to be used.
    """
    model = glad_params(theta_init_offset=0.1, nF=3, H=3)
    optimizer = glad.get_optimizers(model)
    return model, optimizer

def runGLAD(Sb, model_glad):
    """Run the input through the GLAD meta algorithm.
    It executes the following steps in batch mode
    1. Run the GLAD model to get initial good regularization
    2. Calculate the meta-loss
    
    Args:
        Sb (torch.Tensor BxDxD): The input covariance matrix
        metaGLADmodel (dict): Contains the learnable params
    
    Returns:
        loss (torch.scalar): The meta loss 
        predTheta (torch.Tensor BxDxD): The predicted theta
    """
    # 1. Running the GLAD model 
    predTheta = glad.glad(Sb, model_glad)
    # print(f'predTheta: {predTheta}')
    # 2. Calculate the meta-loss
    loss = lossGLADmeta(predTheta, Sb)
    return loss, predTheta


def GLAD_Meta_main(Xb, trueTheta=None, EPOCHS=50):
    """Running the DAG Meta algorithm.
    
    Args:
        Xb (torch.Tensor BxMxD): The input sample matrix
        trueDAG (torch.Tensor BxDxD): The corresponding 
            true DAGs for reporting metrics.
        EPOCHS (int): The number of training epochs
        
    """
    # Calculating the batch covariance
    B, M, D = Xb.shape
    Sb = prepare_data.getCovariance(Xb) # BxDxD
    # optimizer and model for GLAD
    model_glad, optimizer_glad = initGLAD()    
    # Optimizing for the meta loss
    for e in range(EPOCHS):      
        # reset the grads to zero
        optimizer_glad.zero_grad()
        # calculate the loss
        loss, predTheta = runGLAD(Sb, model_glad)
        # calculate the backward gradients
        loss.backward()
        print(f'epoch:{e}/{EPOCHS} loss:{loss.detach().numpy()}')
        # updating the optimizer params with the grads
        optimizer_glad.step()
        # reporting the metrics if true DAGs provided
        if trueTheta is not None and (e+1)%EPOCHS == 0:
            for b in range(B):
                compare_theta = reportMetrics(
                    trueTheta[b].detach().numpy(), 
                    predTheta[b].detach().numpy()
                )
                print(f'Batch:{b} - {compare_theta}')

In [5]:
GLAD_Meta_main(Xb, trueTheta)

epoch:0/50 loss:15.407833099365234
epoch:1/50 loss:14.706966400146484
epoch:2/50 loss:14.441723823547363
epoch:3/50 loss:14.288818359375
epoch:4/50 loss:14.00145149230957
epoch:5/50 loss:13.23261833190918
epoch:6/50 loss:12.17892837524414
epoch:7/50 loss:11.178218841552734
epoch:8/50 loss:10.343488693237305
epoch:9/50 loss:9.343017578125
epoch:10/50 loss:9.262606620788574
epoch:11/50 loss:9.162870407104492
epoch:12/50 loss:9.224103927612305
epoch:13/50 loss:8.863062858581543
epoch:14/50 loss:8.64699649810791
epoch:15/50 loss:8.55723762512207
epoch:16/50 loss:8.505102157592773
epoch:17/50 loss:8.465518951416016
epoch:18/50 loss:8.418127059936523
epoch:19/50 loss:8.381132125854492
epoch:20/50 loss:8.352457046508789
epoch:21/50 loss:8.3326997756958
epoch:22/50 loss:8.317798614501953
epoch:23/50 loss:8.303499221801758
epoch:24/50 loss:8.289525032043457
epoch:25/50 loss:8.27793025970459
epoch:26/50 loss:8.266629219055176
epoch:27/50 loss:8.256170272827148
epoch:28/50 loss:8.246692657470703
